In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.colors as pc
from IPython.display import clear_output

In [2]:
nfl_main_df = pd.read_csv("nfl_main.csv")

In [3]:
team_colors = {
    'ARI': 'rgb(151, 35, 63)','ATL': 'rgb(0, 0, 0)','BAL': 'rgb(26, 25, 95)','BUF': 'rgb(0, 51, 141)',
    'CAR': 'rgb(0, 133, 202)','CHI': 'rgb(11, 22, 42)','CIN': 'rgb(255, 60, 0)','CLE': 'rgb(49, 29, 0)',
    'DAL': 'rgb(0, 34, 68)','DEN': 'rgb(0, 34, 68)','DET': 'rgb(0, 118, 182)','GB': 'rgb(24, 48, 40)',
    'HOU': 'rgb(3, 32, 47)','IND': 'rgb(0, 44, 95)','JAX': 'rgb(0, 103, 120)',
    'KC': 'rgb(227, 24, 55)','LA': 'rgb(0, 53, 148)','LAC': 'rgb(0, 128, 198)',
    'LV': 'rgb(0, 0, 0)','MIA': 'rgb(0, 142, 151)','MIN': 'rgb(79, 38, 131)',
    'NE': 'rgb(0, 34, 68)','NO': 'rgb(211, 188, 141)','NYG': 'rgb(1, 35, 82)',
    'NYJ': 'rgb(18, 87, 64)','PHI': 'rgb(0, 76, 84)',
    'PIT': 'rgb(0, 0, 0)','SEA': 'rgb(0, 21, 50)','SF': 'rgb(170, 0, 0)',
    'TB': 'rgb(213, 10, 10)','TEN': 'rgb(68, 149, 209)','WAS': 'rgb(63, 16, 16)'
}


In [17]:


# Function to respond to dropdown selection and return plot
def game_dashboard(week, season, team, stats): 
    with output:
        # Clear previous plot
        clear_output(wait=True)

        # Filter data based on week, season, and team
        week_df = nfl_main_df[(nfl_main_df["week"] == week) & (nfl_main_df["season"] == season)]
        if week_df.empty:
            print("No data for the selected filters.")
            return

        try:
            game_id = week_df[week_df["team"] == team]["game_id"].values[0]
            game_df = nfl_main_df[nfl_main_df["game_id"] == game_id]
            
            # Determine grid size based on number of stats
            num_stats = len(stats)
            ncols = 2
            nrows = int(np.ceil(num_stats / ncols))

            # Create a subplot layout
            fig = sp.make_subplots(rows=nrows, cols=ncols, subplot_titles=stats)

            # Plot each statistic
            for i, s in enumerate(stats):
                filtered_data = game_df[game_df[s] > 0].sort_values(s, ascending=False)
                if not filtered_data.empty:
                    # Assign color based on team
                    colors = [team_colors.get(t, 'grey') for t in filtered_data['team']]
                    
                    trace = go.Bar(
                        x=filtered_data["player_display_name"],
                        y=filtered_data[s],
                        name=s,
                        marker=dict(color=colors),  # Apply team color
                        text=filtered_data["team"],
                        hoverinfo="text+y"
                    )
                    row = i // ncols + 1
                    col = i % ncols + 1
                    fig.add_trace(trace, row=row, col=col)
                else:
                    row = i // ncols + 1 
                    col = i % ncols + 1 
                    fig.add_annotation(
                        x=0.8, y=0.8, 
                        text="No Data", 
                        showarrow=False, 
                        xref=f"x{i+1}", 
                        yref=f"y{i+1}",
                        font=dict(color="red"),
                        row=row, col=col
                    )

            # Update layout and axis labels
            fig.update_layout(
                height=400 * nrows, 
                width=800, 
                title_text="Game Statistics",
                showlegend=False
            )
            fig.update_xaxes(tickangle=-70)

            # Display the plot
            fig.show()

        except IndexError:
            print("No matching game data for the selected week, season, and team.")


In [18]:
# Create four dropdown widgets
dropdown1 = widgets.Dropdown(
    options=list(nfl_main_df["week"].sort_values().unique()),
    value=1,
    description='Week: ',
)

dropdown2 = widgets.Dropdown(
    options=list(nfl_main_df["season"].sort_values().unique()),
    value=2024,
    description='Season: ',
)

dropdown3 = widgets.Dropdown(
    options=list(nfl_main_df["team"].sort_values().unique()),
    value='PHI',
    description='Team: ',
)

dropdown4 = widgets.SelectMultiple(
    options=['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs'],
    value=['completions', 'passing_tds', 'receptions', 'receiving_tds'],  # Default selection
    description='Stats:',
)

# Output widget to control where the plot will be displayed
output = widgets.Output()



# Function to update the plot based on dropdown values
def update_plot(*args):
    game_dashboard(dropdown1.value, dropdown2.value, dropdown3.value, list(dropdown4.value))

# Observe dropdown value changes
dropdown1.observe(update_plot, names='value')
dropdown2.observe(update_plot, names='value')
dropdown3.observe(update_plot, names='value')
dropdown4.observe(update_plot, names='value')

# Display dropdowns and output
display(dropdown1, dropdown2, dropdown3, dropdown4, output)

# Call the plot function initially
update_plot()

Dropdown(description='Week: ', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

Dropdown(description='Season: ', index=5, options=(2019, 2020, 2021, 2022, 2023, 2024), value=2024)

Dropdown(description='Team: ', index=25, options=('ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL…

SelectMultiple(description='Stats:', index=(0, 3, 24, 27), options=('completions', 'attempts', 'passing_yards'…

Output()

In [19]:



# Function to respond to dropdown selection and return plot
def season_dashboard(season, stats): 
    with output:
        # Clear previous plot
        clear_output(wait=True)

        # Filter data based on season
        season_df = nfl_main_df[(nfl_main_df["season"] == season)].groupby(["player_display_name","team"])[stats].sum().reset_index()
        if season_df.empty:
            print("No data for the selected filters.")
            return

        try:

            # Determine grid size based on number of stats
            num_stats = len(stats)
            ncols = 2
            nrows = int(np.ceil(num_stats / ncols))

            # Create a subplot layout
            fig = sp.make_subplots(rows=nrows, cols=ncols, subplot_titles=stats)

            # Plot each statistic
            for i, s in enumerate(stats):
                filtered_data = season_df[season_df[s] > 0].sort_values(s, ascending=False)[:10]
                if not filtered_data.empty:
                    # Assign color based on team
                    colors = [team_colors.get(t, 'grey') for t in filtered_data['team']]
                    
                    trace = go.Bar(
                        x=filtered_data["player_display_name"],
                        y=filtered_data[s],
                        name=s,
                        marker=dict(color=colors),  # Apply team color
                        text=filtered_data["team"],
                        hoverinfo="text+y"
                    )
                    row = i // ncols + 1
                    col = i % ncols + 1
                    fig.add_trace(trace, row=row, col=col)
                else:
                    row = i // ncols + 1 
                    col = i % ncols + 1 
                    fig.add_annotation(
                        x=0.8, y=0.8, 
                        text="No Data", 
                        showarrow=False, 
                        xref=f"x{i+1}", 
                        yref=f"y{i+1}",
                        font=dict(color="red"),
                        row=row, col=col
                    )

            # Update layout and axis labels
            fig.update_layout(
                height=400 * nrows, 
                width=800, 
                title_text="Game Statistics",
                showlegend=False
            )
            fig.update_xaxes(tickangle=-70)

            # Display the plot
            fig.show()

        except IndexError:
            print("No matching game data for the selected week, season, and team.")


In [20]:
# Create four dropdown widgets

dropdown2 = widgets.Dropdown(
    options=list(nfl_main_df["season"].sort_values().unique()),
    value=2024,
    description='Season: ',
)

dropdown4 = widgets.SelectMultiple(
    options=['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs'],
    value=['completions', 'passing_tds', 'receptions', 'receiving_tds'],  # Default selection
    description='Stats:',
)

# Output widget to control where the plot will be displayed
output = widgets.Output()



# Function to update the plot based on dropdown values
def update_plot(*args):
    season_dashboard(dropdown2.value, list(dropdown4.value))

# Observe dropdown value changes
dropdown2.observe(update_plot, names='value')
dropdown4.observe(update_plot, names='value')

# Display dropdowns and output
display(dropdown2, dropdown4, output)

# Call the plot function initially
update_plot()

Dropdown(description='Season: ', index=5, options=(2019, 2020, 2021, 2022, 2023, 2024), value=2024)

SelectMultiple(description='Stats:', index=(0, 3, 24, 27), options=('completions', 'attempts', 'passing_yards'…

Output()